Let's evaluate 2 vectors representative of:

* The users abilities
* The problem dfficulties

We are going to use 2 different methods and try to comare them.

In order to validate the methods, we need first to apply them to a small dummy dataset, automaticly generated

In [1]:
%matplotlib inline
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dummy import *

# Dataset generation

the function *generate_submissions* generete a dataframe of submission for a specific number of users and problems.

This will the basis data for our analysis

In [2]:
subs = generate_submissions(users=4, pbls=4)

Fro that dataframe, we can generate a matrix representing the view of  which user maaged to solved which problems in the given submission history.

In [3]:
bm = bianary_mat(subs)
bm

,U00000,U00001,U00002,U00003
P00000,1,0,1,1
P00001,0,1,0,1
P00002,1,1,1,0
P00003,0,0,1,1


> The matrice is binary. We just need to know if a user solved problem, no matter in how many tries. We should also take account of it in a 
subsequent analysis.

# Linear evaluation

First of all, we want to evaluate a level o difficulty for each problem in a static way.

We can consider for example that a specific problem is less difficult if a lot of users solved it.

In [4]:
def pblDiff(v):
    '''
    v is the a binary verctor
    '''   
    return 1.0 - (float(v.sum()) / len(v))

In [5]:
PbDifficulties = [pblDiff(x) for x in bm.values]
normalize(PbDifficulties)
PbDifficulties = pd.DataFrame(PbDifficulties, index=bm.index, 
                              columns=['diff'])
PbDifficulties

,diff
P00000,0.316228
P00001,0.632456
P00002,0.316228
P00003,0.632456


Now we cant compute for each user his ability to resove problems, using a linear function that sum the every solved pbl difficulty.

In [6]:
plop = PbDifficulties.join(bm)
plop

,diff,U00000,U00001,U00002,U00003
P00000,0.316228,1,0,1,1
P00001,0.632456,0,1,0,1
P00002,0.316228,1,1,1,0
P00003,0.632456,0,0,1,1


In [7]:
print plop[plop.U00000 > 0]['diff'].sum()
print plop[plop.U00001 > 0]['diff'].sum()
print plop[plop.U00002 > 0]['diff'].sum()
print plop[plop.U00003 > 0]['diff'].sum()

0.632455532034
0.948683298051
1.26491106407
1.58113883008


In [8]:
def Abilities(mat, pblslev):
    df = pblslev.join(mat)
    return [df[df[i] > 0 ]['diff'].sum() for i in bm]
    

In [9]:
usrAb = Abilities(bm, PbDifficulties)
normalize(usrAb)
usrAb = pd.DataFrame(usrAb, index=bm.columns, columns=['abl']).T
usrAb

,U00000,U00001,U00002,U00003
abl,0.272166,0.408248,0.544331,0.680414


In [10]:
plop.append(usrAb)

,U00000,U00001,U00002,U00003,diff
P00000,1.000000,0.000000,1.000000,1.000000,0.316228
P00001,0.000000,1.000000,0.000000,1.000000,0.632456
P00002,1.000000,1.000000,1.000000,0.000000,0.316228
P00003,0.000000,0.000000,1.000000,1.000000,0.632456
abl,0.272166,0.408248,0.544331,0.680414,NaN


In [12]:
bm

,U00000,U00001,U00002,U00003
P00000,1,0,1,1
P00001,0,1,0,1
P00002,1,1,1,0
P00003,0,0,1,1


In [13]:
bmm = bm.as_matrix()
U, s, V = np.linalg.svd(bmm, full_matrices=False)

In [14]:
print U

[[-0.62191759 -0.10930527 -0.43477072  0.64206331]
 [-0.34063033 -0.3411985   0.85503794  0.19095729]
 [-0.54369869  0.77215862  0.15616783 -0.28943809]
 [-0.44897798 -0.52479235 -0.23557623 -0.68375113]]


In [15]:
Uv = pd.DataFrame(U)[0]
print Uv

0   -0.621918
1   -0.340630
2   -0.543699
3   -0.448978
Name: 0, dtype: float64


In [17]:
print s
s[0]

[ 2.59615468  1.26307741  1.18264432  0.51572159]


2.5961546760086334

In [18]:
print V
V[0]

[[-0.44897798 -0.34063033 -0.62191759 -0.54369869]
 [ 0.52479235  0.3411985   0.10930527 -0.77215862]
 [-0.23557623  0.85503794 -0.43477072  0.15616783]
 [ 0.68375113 -0.19095729 -0.64206331  0.28943809]]


array([-0.44897798, -0.34063033, -0.62191759, -0.54369869])